In [1]:
%%capture
pip install transformers

In [2]:
import pandas as pd
import torch 
import numpy as np
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import SGD

/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Read CSV Data

In [8]:
df = pd.read_csv('entities.csv')
df.head()

,Utterance,Labels
0,$ .,O O
1,$ 13 - 18k offer is still low for your fujifil...,B-pr_curr B-pr_val I-pr_val I-pr_val O O O O O...
2,"$ 5,150 totally .",B-pr_curr B-pr_val B-pr_size O
3,$ 9500 per unit is too much .,B-pr_curr B-pr_val B-pr_size I-pr_size O O O O
4,$ 950k - $ 1.3 m is okay .,B-pr_curr B-pr_val O B-pr_curr B-pr_val I-pr_v...


# Initialize Tokenizer

In [5]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Create Dataset Class 

In [9]:
label_all_tokens = False

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):
        lb = [i.split() for i in df['Labels'].values.tolist()]
        txt = df['Utterance'].values.tolist()
        self.texts = [tokenizer(str(i), padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
        self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

    def __len__(self):
        return len(self.labels)

    def get_batch_data(self, idx):
        return self.texts[idx]

    def get_batch_labels(self, idx):
        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):
        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

# Split Data and Define Unique Labels

In [11]:
labels = [i.split() for i in df['Labels'].values.tolist()]
unique_labels = set()

for lb in labels:
        [unique_labels.add(i) for i in lb if i not in unique_labels]
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}

df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.98 * len(df)), int(.99 * len(df))])

print(len(df_train),len(df_val), len(df_test))

6724 69 69


In [12]:
print(labels)

[['O', 'O'], ['B-pr_curr', 'B-pr_val', 'I-pr_val', 'I-pr_val', 'O', 'O', 'O', 'O', 'O', 'O', 'B-eq_mm', 'I-eq_mm', 'I-eq_mm', 'I-eq_mm', 'O'], ['B-pr_curr', 'B-pr_val', 'B-pr_size', 'O'], ['B-pr_curr', 'B-pr_val', 'B-pr_size', 'I-pr_size', 'O', 'O', 'O', 'O'], ['B-pr_curr', 'B-pr_val', 'O', 'B-pr_curr', 'B-pr_val', 'I-pr_val', 'O', 'B-act_acc', 'O'], ['B-pr_curr', 'B-pr_val', 'I-pr_val', 'O', 'O', 'O', 'B-act_acc', 'O', 'O'], ['B-pr_curr', 'B-pr_val', 'O', 'O', 'B-eq_mm', 'O', 'O', 'B-eq_type', 'O'], ['B-pr_curr', 'B-pr_val', 'B-cat_oth', 'O'], ['B-pr_curr', 'B-pr_val', 'B-cat_oth', 'I-cat_oth', 'O'], ['B-pr_curr', 'B-pr_val', 'I-pr_val', 'B-cat_oth', 'O'], ['B-pr_curr', 'B-pr_val', 'O', 'B-pr_curr', 'B-pr_val', 'I-pr_val', 'O', 'O', 'B-eq_type', 'O'], ['B-pr_curr', 'B-pr_val', 'O', 'O', 'B-cond_asis', 'I-cond_asis', 'I-cond_asis', 'O'], ['B-pr_curr', 'B-pr_val', 'O', 'O', 'O'], ['B-pr_curr', 'B-pr_val', 'B-pr_size', 'I-pr_size', 'O'], ['B-pr_curr', 'B-pr_val', 'O', 'B-cat_rang', 'O'],

# Build Model

In [13]:
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(unique_labels))

    def forward(self, input_ids, attention_mask, labels):

        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, labels=labels, return_dict=False)

        return output

# Model Training

In [14]:
def train_loop(model, df_train, df_val, epochs=5, batch_size=2, learninig_rate=5e-3):

    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=learninig_rate)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):

                logits_clean = logits[i][train_label[i] != -100]
                label_clean = train_label[i][train_label[i] != -100]

                predictions = logits_clean.argmax(dim=1)
                acc = (predictions == label_clean).float().mean()
                total_acc_train += acc
                total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

                logits_clean = logits[i][val_label[i] != -100]
                label_clean = val_label[i][val_label[i] != -100]

                predictions = logits_clean.argmax(dim=1)
                acc = (predictions == label_clean).float().mean()
                total_acc_val += acc
                total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')



In [15]:
LEARNING_RATE = 5e-3
EPOCHS = 10
BATCH_SIZE = 2


model = BertModel()
train_loop(model, df_train, df_val, EPOCHS, BATCH_SIZE, LEARNING_RATE)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

Epochs: 1 | Loss:  1.024 | Accuracy:  0.774 | Val_Loss:  0.683 | Accuracy:  0.821


100%|██████████| 3362/3362 [02:54<00:00, 19.23it/s]


Epochs: 2 | Loss:  0.506 | Accuracy:  0.868 | Val_Loss:  0.542 | Accuracy:  0.842


100%|██████████| 3362/3362 [02:54<00:00, 19.23it/s]


...

Epochs: 8 | Loss:  0.123 | Accuracy:  0.965 | Val_Loss:  0.479 | Accuracy:  0.874


100%|██████████| 3362/3362 [02:54<00:00, 19.24it/s]


Epochs: 9 | Loss:  0.106 | Accuracy:  0.970 | Val_Loss:  0.523 | Accuracy:  0.865


100%|██████████| 3362/3362 [02:54<00:00, 19.24it/s]


Epochs: 10 | Loss:  0.093 | Accuracy:  0.974 | Val_Loss:  0.551 | Accuracy:  0.881


# Evaluate Model

In [16]:
def evaluate(model, df_test):

    test_dataset = DataSequence(df_test)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0.0

    for test_data, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_data['attention_mask'].squeeze(1).to(device)

            input_id = test_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, test_label)

            for i in range(logits.shape[0]):

                logits_clean = logits[i][test_label[i] != -100]
                label_clean = test_label[i][test_label[i] != -100]

                predictions = logits_clean.argmax(dim=1)
                acc = (predictions == label_clean).float().mean()
                total_acc_test += acc

    val_accuracy = total_acc_test / len(df_test)
    print(f'Test Accuracy: {val_accuracy: .3f}')


In [17]:
evaluate(model, df_test)

Test Accuracy:  0.904


# Predict One Sentence

In [18]:
from spacy import displacy
import typing as tp

In [19]:
def ner_render(tokens: tp.Sequence[str], ner_tags: tp.Sequence[str], title: tp.Optional[str] = None, **kwargs):
    pos = 0
    ents = []
    for word, tag in zip(tokens, ner_tags):
        if tag.startswith('B'):
            ents.append({
                "start": pos,
                "end": pos + len(word),
                "label": tag.split("-")[1]
            })
        elif tag.startswith('I'):
            ents[-1]["end"] = pos + len(word)
        pos += (len(word) + 1)
    displacy.render({
        "text": " ".join(tokens),
        "ents": ents,
        "title": title
    }, style="ent", manual=True, jupyter=True)

In [20]:
def align_word_ids(texts):
    
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            # label_ids.append(-100)
            label_ids.append(0)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

In [68]:
def evaluate_one_text(model, sentence):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    encoded = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
    
    mask = encoded['attention_mask'].to(device)
    input_id = encoded['input_ids'].to(device)

    logits = model(input_id, mask, None)

    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits_clean = logits[0][label_ids != 0]
    # predictions = logits_clean.argmax(dim=1).tolist()
    predictions = logits_clean.argmax(dim=1).tolist()

    # print(predictions)
    prediction_labels = [ids_to_labels[i] for i in predictions]

    tokens = tokenizer.convert_ids_to_tokens(input_id[0])
    tokens = tokens[1:1+len(prediction_labels)]
    
    ner_render(tokens, prediction_labels)

In [90]:
evaluate_one_text(model, "500 $ for this.")

[106, 79, 89, 89, 89]


In [47]:
onnxfile = "ner_classifier.onnx"
modified_onnxfile = "modified_ner_model.onnx"

In [48]:
path = './model-dict-ent.dat'
torch.save(model.state_dict(), path)

In [49]:
path = './model-dict-ent.dat'
model1 = BertModel()
model1.load_state_dict(torch.load(path))
model1.eval()
evaluate_one_text(model1, "what is the price for this amat/applied materials unit.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [93]:
input_names = ['input_id', 'mask', 'label']
output_names = ['tags']

txt = "Do you have this equipment?"
encoded = tokenizer(txt, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
dummy_inputs = {'input_id': encoded['input_ids'], 'mask': encoded['attention_mask'], "label":None }

model1.to("cpu")

model1.eval()

torch.onnx.export(model1,                 
                  args=dummy_inputs,
                  input_names=input_names,
                  output_names=output_names,
                  f=onnxfile,
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True  # whether to execute constant folding for optimization
                #   dynamic_axes={'input_ids' : {0 : 'batch_size'},    # variable length axes
                #                 'probs' : {0 : 'batch_size'}}
                #   export_params=True,
                #   verbose=True
)

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [59]:
evaluate_one_text(model, 'what is the price for this amat/applied materials unit.')

(tensor([[[ 0.3857,  0.1416, -1.5498,  ..., -0.4041, -0.0523,  0.0827],
         [ 1.6317, -0.4771, -0.5347,  ..., -0.8656, -1.3691, -1.3907],
         [ 2.5318, -0.9031, -0.2257,  ..., -1.0531, -1.9201, -1.8763],
         ...,
         [-1.3057,  0.1364, -0.8737,  ..., -0.6801,  0.1459, -0.4902],
         [-0.3633, -0.5076,  0.0870,  ..., -0.2998, -0.0826, -0.9674],
         [-1.5533,  0.0405, -0.5586,  ..., -0.1981, -0.5346, -1.2566]]],
       device='cuda:0', grad_fn=<ViewBackward0>),)


In [24]:
evaluate_one_text(model, 'I want to buy amat / applied materials unit.')

In [25]:
evaluate_one_text(model, "kla tencor?")

In [26]:
evaluate_one_text(model, "Do you have this equipment?")

In [27]:
evaluate_one_text(model, "What is the whether today?")

In [28]:
evaluate_one_text(model, "price?")

In [29]:
evaluate_one_text(model, "is it still deinstalled?")

In [30]:
evaluate_one_text(model, "we are going to remove this unit from production next week.")

In [31]:
evaluate_one_text(model, "we can offer $ 70,000 for the picosun ald in an effort to try to come to an agreement this week .")

In [32]:
evaluate_one_text(model, "$ 19k / ea is much beyond my budget even it is new .")

In [33]:
evaluate_one_text(model, "my customer budget for candella candela cs20r is 125k based on working condition mike .")

In [34]:
evaluate_one_text(model, "we have a dual turn with gantry loader already .")

In [35]:
evaluate_one_text(model, "but it is still being used not heavily but still being used w / back side alignment .")

In [36]:
evaluate_one_text(model, "We do not have approval to purchase anything yet")

In [37]:
evaluate_one_text(model, "who needs this equipment")

In [38]:
evaluate_one_text(model, "pls help me to find FEI FIB ,200,450") ## not correct

In [41]:
evaluate_one_text(model, "april 16 .") ## not correct

In [42]:
evaluate_one_text(model, "sorry we are buying not selling.")

In [44]:
evaluate_one_text(model, "5k £ for this.")